# Word Embedding

In [1]:
"""
    @Project: nlp-learning-tutorials
    @File   : Embedding.ipynb
    @Author : Hivan Du
    @E-mail : doo@hivan.me
    @Wechat : 坍缩的奇点
    @Date   : 2023-12-24 15:35
"""
import os
print(os.getcwd())
import gensim

print(gensim.__version__)

/Volumes/Library/SynologyDrive/ChaHengAI/AI_Cheats/Core BI
4.3.3


In [2]:
import jieba
import os
# utils 为自定义的一套程序
from utils import files_processing
from gensim.models import word2vec
import multiprocessing

data_path = os.environ.get('DATA_PATH') + 'AI_Cheats/'

/Users/bindu/miniforge3/envs/mlo/lib/python3.10/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


ModuleNotFoundError: No module named 'utils'

## Word segment

In [3]:
# 源文件所在目录
source_folder = data_path + 'journey_to_the_west/source'
segment_folder = data_path + 'journey_to_the_west/segment'

NameError: name 'data_path' is not defined

In [ ]:
# 字词分割，对整个文件内容进行字词分割
def segment_lines(file_list,segment_out_dir,stopwords=[]):
    for i,file in enumerate(file_list):
        segment_out_name=os.path.join(segment_out_dir,'segment_{}.txt'.format(i))
        with open(file, 'rb') as f:
            document = f.read()
            document_cut = jieba.cut(document)
            sentence_segment=[]
            for word in document_cut:
                if word not in stopwords:
                    sentence_segment.append(word)
            result = ' '.join(sentence_segment)
            result = result.encode('utf-8')
            with open(segment_out_name, 'wb') as f2:
                f2.write(result)

In [ ]:
# 对 source 中的 txt 文件进行分词，输出到 segment 目录中
file_list=files_processing.get_files_list(source_folder, postfix='*.txt')
segment_lines(file_list, segment_folder)

Building prefix dict from the default dictionary ...


Dumping model to file cache /var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/jieba.cache
Loading model cost 0.470 seconds.
Prefix dict has been built successfully.


## Word similarity

In [ ]:
# 如果目录中有多个文件，可以使用 PathLineSentences
segment_folder = data_path + 'journey_to_the_west/segment'
sentences = word2vec.PathLineSentences(segment_folder)

In [ ]:
# 设置模型参数，进行训练
model = word2vec.Word2Vec(sentences, vector_size=100, window=3, min_count=1)
print(model.wv.similarity('孙悟空', '猪八戒'))
print(model.wv.similarity('孙悟空', '孙行者'))
print(model.wv.most_similar(positive=['孙悟空', '唐僧'], negative=['孙行者']))

0.92123955
0.97135127
[('三怪', 0.9665513634681702), ('者', 0.9608936905860901), ('菩萨', 0.9601831436157227), ('老', 0.9580262899398804), ('长老', 0.9579092264175415), ('银角', 0.9577627182006836), ('师父', 0.9573383927345276), ('有事', 0.9552664160728455), ('大王', 0.9537328481674194), ('悟净', 0.952475905418396)]


In [ ]:
# 设置模型参数，进行训练
model2 = word2vec.Word2Vec(sentences, vector_size=128, window=5, min_count=5, workers=multiprocessing.cpu_count())

In [ ]:
# 保存模型
model2.save(data_path + 'models/word2Vec.model')

In [ ]:
print(model2.wv.similarity('孙悟空', '猪八戒'))
print(model2.wv.similarity('孙悟空', '孙行者'))
print(model2.wv.most_similar(positive=['孙悟空', '唐僧'], negative=['孙行者']))

0.91111225
0.9480323
[('菩萨', 0.9330905675888062), ('吩咐', 0.8870710730552673), ('唐老爷', 0.8861484527587891), ('大仙', 0.8849828243255615), ('送行', 0.8840615153312683), ('三魔', 0.8826634883880615), ('我来', 0.8797548413276672), ('们', 0.8787403106689453), ('弟子', 0.8782590627670288), ('上前', 0.8753783702850342)]
